<a href="https://colab.research.google.com/github/vanshnawander/colab_notebooks_aiml/blob/main/AIML_Module_3_Lab_3_Using_KNN_for_Text_Classification_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MODULE: CLASSIFICATION-1
### LAB-3 : Using KNN for Text Classification



---

## **Section 1: Understanding NLP tools**

In this lab we will be using KNN on a real world NLP application i.e. is text classification. But first look at some NLP techniques for text classification and tools that we use when we want to use python for NLP.

## Section 1.2: Data Cleaning and Preprocessing step

Raw text must be processed and converted into a form so that it is suitable to use with various machine-learning algorithms.  
In case of text, there are lots of things that need to be taken into account.  


1.   Removing numbers from the text
2.   Handling capitalization and punctuation.
3.   Stemming and Lemmatizing text.  

And most importantly, one can't just use words or images directly in algorithms; they need to be converted into vectors- a form that algorithms can understand.



### **NLTK**
NLTK (or Natural Language Tool Kit) is a commonly used library for processing text. We will use this tool in this lab. Lets first install it.


In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
import re
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

def cleanText(text, lemmatize, stemmer):
    """Method for cleaning text from train and test data. Removes numbers, punctuation, and capitalization. Stems or lemmatizes text."""

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^A-Za-z]", " ", text)
    text = text.lower()


    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def get_tag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_result = []
        tokens = word_tokenize(text)  # Generate list of tokens
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
            except:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_result

    if stemmer:
        text_result = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('english')
        for t in tokens:
            text_result.append(snowball_stemmer.stem(t))
        return text_result

In [14]:
sample_text = "cheating"
sample_text_result = cleanText(sample_text, lemmatize=False, stemmer=True)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text)
print(sample_text_result)
sample_text_result = cleanText(sample_text, lemmatize=True, stemmer=False)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text_result)

cheating
cheat
cheat


## Section 1.2: BAG OF WORDS

A bag-of-words model, or BoW for short, is a way of extracting features from text for use in modeling, such as with machine learning algorithms.

The approach is very simple and flexible, and can be used in many ways for extracting features from documents.

A bag-of-words is a representation of text that describes the occurrence of words within a document.
It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

In [15]:
5*12

60

In [16]:
# Functions to convert document(s) to a list of words, with the option of removing stopwords. Returns document-term matrix.

def createBagOfWords(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = CountVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer = CountVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    bag_of_words_train = vectorizer.fit_transform(clean_train).toarray()
    bag_of_words_test = vectorizer.transform(clean_test).toarray()
    return bag_of_words_train, bag_of_words_test


## Section 1.3: TF-IDF
TF-IDF technique is used to find meaning of sentences consisting of words and cancels out the incapabilities of Bag of Words technique which is good for text classification or for helping a machine read words in numbers.

The number of times a term occurs in a document is called its Term frequency (TF).

 Document frequency is the number of documents in which the word is present.  Inverse DF (IDF) is the inverse of the document frequency which measures the informativeness of term *t*.




In [17]:
def createTFIDF(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = TfidfVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    tfidf_train = vectorizer.fit_transform(clean_train).toarray()
    tfidf_test = vectorizer.transform(clean_test).toarray()
    return tfidf_train, tfidf_test

# **Section 2: UNDERSTANDING THE DATA : A REVIEWS DATASET**

Sentiment analysis is the interpretation and classification of emotions (such as positive, negative and neutral) within text data using text analysis techniques.  
Given below is a dataset consisting of reviews along with sentiment class (positive or negative).

In [18]:
# Upload the Reviews CSV file that has been shared with you.
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving reviews.csv to reviews.csv


In [19]:
import pandas as pd
df = pd.read_csv('reviews.csv')

In [20]:
df

,sentence,sentiment
0,Not sure who was more lost - the flat characte...,0
1,Attempting artiness with black & white and cle...,0
2,Very little music or anything to speak of.,0
3,The best scene in the movie was when Gerardo i...,1
4,"The rest of the movie lacks art, charm, meanin...",0
...,...,...
994,I just got bored watching Jessice Lange take h...,0
995,"Unfortunately, any virtue in this film's produ...",0
996,"In a word, it is embarrassing.",0
997,Exceptionally bad!,0


In [21]:
df = df.dropna()

In [22]:
df

,sentence,sentiment
0,Not sure who was more lost - the flat characte...,0
1,Attempting artiness with black & white and cle...,0
2,Very little music or anything to speak of.,0
3,The best scene in the movie was when Gerardo i...,1
4,"The rest of the movie lacks art, charm, meanin...",0
...,...,...
994,I just got bored watching Jessice Lange take h...,0
995,"Unfortunately, any virtue in this film's produ...",0
996,"In a word, it is embarrassing.",0
997,Exceptionally bad!,0


In [23]:
df.to_csv('reviews.csv', index=False)

In [24]:
df

,sentence,sentiment
0,Not sure who was more lost - the flat characte...,0
1,Attempting artiness with black & white and cle...,0
2,Very little music or anything to speak of.,0
3,The best scene in the movie was when Gerardo i...,1
4,"The rest of the movie lacks art, charm, meanin...",0
...,...,...
994,I just got bored watching Jessice Lange take h...,0
995,"Unfortunately, any virtue in this film's produ...",0
996,"In a word, it is embarrassing.",0
997,Exceptionally bad!,0


# **Section 3: KNN MODEL**

Given below are two KNN models; in the first case we are using Bag-of-Words and in the second case we are using TF-IDF.
Note the different metrics and parameters used in each.

In [54]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 1: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn(neigh=5,weigh='uniform',algo='auto',metri='euclidian'):
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    #knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)
    #knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)
    #knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='kd_tree', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)
    #knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='ball_tree', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)
    knn = neighbors.KNeighborsClassifier(n_neighbors=neigh, weights=weigh, algorithm=algo, leaf_size=30, p=2, metric=metri, metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print(f"the metrics used are n_neighbours={neigh}, weights={weigh}, algorithm={algo}, metrics={metri}")
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test



In [78]:

def tfidf_knn(neigh=5,weigh='uniform',algo='auto',metri='euclidian'):
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"],
                                                        test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=neigh, weights=weigh, algorithm=algo, leaf_size=30, p=2, metric=metri, metric_params=None, n_jobs=1)
    print(f"the metrics used are n_neighbours={neigh}, weights={weigh}, algorithm={algo}, metrics={metri}")
    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

Note: Cross-validation will be discussed in detail in the upcoming lab session.

In [41]:
import warnings
warnings.filterwarnings('ignore')

In [74]:
from random import choice
import sklearn
neighbours=[3,5,7,9,11]
weights=['uniform','distance']
algorithm=['ball_tree', 'kd_tree', 'brute']
metric=sklearn.neighbors.BallTree.valid_metrics
#metric.remove("mahalanobis")
#metric.remove("haversine")
brute_metric=sklearn.neighbors.VALID_METRICS['brute']
try:
  for i in range(100):
    algo=choice(algorithm)
    if(algo=='ball_tree'):
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(metric))
    elif(algo=='kd_tree'):
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(sklearn.neighbors.KDTree.valid_metrics))
    else:
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(brute_metric))
except:
  pass

the metrics used are n_neighbours=9, weights=uniform, algorithm=brute, metrics=yule
KNN with BOW accuracy = 70.68062827225131%
Cross Validation Accuracy: 0.69
[0.69019608 0.68627451 0.70472441]


the metrics used are n_neighbours=5, weights=uniform, algorithm=kd_tree, metrics=manhattan
KNN with BOW accuracy = 63.87434554973822%
Cross Validation Accuracy: 0.62
[0.63529412 0.60784314 0.61811024]


the metrics used are n_neighbours=9, weights=distance, algorithm=brute, metrics=russellrao
KNN with BOW accuracy = 74.86910994764398%
Cross Validation Accuracy: 0.69
[0.68235294 0.69411765 0.70472441]




In [75]:
try:
  for i in range(100):
    algo=choice(algorithm)
    if(algo=='ball_tree'):
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(metric))
    elif(algo=='kd_tree'):
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(sklearn.neighbors.KDTree.valid_metrics))
    else:
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(brute_metric))
except:
  pass

the metrics used are n_neighbours=11, weights=uniform, algorithm=brute, metrics=l1
KNN with BOW accuracy = 64.92146596858639%
Cross Validation Accuracy: 0.63
[0.64313725 0.6        0.65354331]


the metrics used are n_neighbours=11, weights=uniform, algorithm=ball_tree, metrics=chebyshev
KNN with BOW accuracy = 49.21465968586388%
Cross Validation Accuracy: 0.50
[0.47843137 0.50980392 0.52362205]


the metrics used are n_neighbours=11, weights=uniform, algorithm=ball_tree, metrics=russellrao
KNN with BOW accuracy = 71.20418848167539%
Cross Validation Accuracy: 0.70
[0.66666667 0.70588235 0.71653543]


the metrics used are n_neighbours=5, weights=distance, algorithm=brute, metrics=l2
KNN with BOW accuracy = 63.35078534031413%
Cross Validation Accuracy: 0.62
[0.60784314 0.59215686 0.66141732]


the metrics used are n_neighbours=3, weights=distance, algorithm=ball_tree, metrics=minkowski
KNN with BOW accuracy = 63.87434554973822%
Cross Validation Accuracy: 0.60
[0.61568627 0.6        0.598

In [77]:
try:
  for i in range(50):
    algo=choice(algorithm)
    if(algo=='ball_tree'):
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(metric))
    elif(algo=='kd_tree'):
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(sklearn.neighbors.KDTree.valid_metrics))
    else:
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(brute_metric))
except:
  pass

the metrics used are n_neighbours=9, weights=uniform, algorithm=brute, metrics=l2
KNN with BOW accuracy = 59.68586387434554%
Cross Validation Accuracy: 0.63
[0.61960784 0.61176471 0.66535433]


the metrics used are n_neighbours=11, weights=distance, algorithm=ball_tree, metrics=sokalsneath
KNN with BOW accuracy = 71.72774869109948%
Cross Validation Accuracy: 0.69
[0.71764706 0.69803922 0.6496063 ]


the metrics used are n_neighbours=7, weights=uniform, algorithm=kd_tree, metrics=chebyshev
KNN with BOW accuracy = 45.0261780104712%
Cross Validation Accuracy: 0.51
[0.51764706 0.51372549 0.51181102]


the metrics used are n_neighbours=5, weights=uniform, algorithm=kd_tree, metrics=manhattan
KNN with BOW accuracy = 63.87434554973822%
Cross Validation Accuracy: 0.62
[0.63529412 0.60784314 0.61811024]


the metrics used are n_neighbours=7, weights=uniform, algorithm=brute, metrics=chebyshev
KNN with BOW accuracy = 49.21465968586388%
Cross Validation Accuracy: 0.52
[0.54509804 0.51764706 0.507

None


In [65]:
import sklearn
print(sklearn.neighbors.BallTree.valid_metrics)
print(sklearn.neighbors.KDTree.valid_metrics)
print(sklearn.neighbors.VALID_METRICS['brute'])

['euclidean', 'l2', 'minkowski', 'p', 'manhattan', 'cityblock', 'l1', 'chebyshev', 'infinity', 'wminkowski', 'hamming', 'canberra', 'braycurtis', 'matching', 'jaccard', 'dice', 'kulsinski', 'rogerstanimoto', 'russellrao', 'sokalmichener', 'sokalsneath', 'pyfunc']
['euclidean', 'l2', 'minkowski', 'p', 'manhattan', 'cityblock', 'l1', 'chebyshev', 'infinity']
['braycurtis', 'canberra', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'haversine', 'jaccard', 'kulsinski', 'l1', 'l2', 'mahalanobis', 'manhattan', 'matching', 'minkowski', 'nan_euclidean', 'precomputed', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule']


In [80]:
try:
  for i in range(100):
    algo=choice(algorithm)
    if(algo=='ball_tree'):
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(metric))
    elif(algo=='kd_tree'):
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(sklearn.neighbors.KDTree.valid_metrics))
    else:
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(brute_metric))
    print("\n")
except:
  pass

the metrics used are n_neighbours=3, weights=distance, algorithm=brute, metrics=sokalmichener
KNN with TFIDF accuracy = 61.78010471204188%
Cross Validation Accuracy: 0.60
[0.61568627 0.6        0.58661417]


the metrics used are n_neighbours=3, weights=uniform, algorithm=kd_tree, metrics=l2
KNN with TFIDF accuracy = 71.20418848167539%
Cross Validation Accuracy: 0.71
[0.6745098  0.76470588 0.70472441]


the metrics used are n_neighbours=7, weights=distance, algorithm=brute, metrics=nan_euclidean
KNN with TFIDF accuracy = 70.15706806282722%
Cross Validation Accuracy: 0.74
[0.70980392 0.74509804 0.7519685 ]


the metrics used are n_neighbours=7, weights=uniform, algorithm=ball_tree, metrics=manhattan
KNN with TFIDF accuracy = 66.49214659685863%
Cross Validation Accuracy: 0.63
[0.62745098 0.62352941 0.64173228]


the metrics used are n_neighbours=3, weights=distance, algorithm=brute, metrics=jaccard
KNN with TFIDF accuracy = 72.77486910994764%
Cross Validation Accuracy: 0.68
[0.70588235 0.

In [82]:
try:
  for i in range(100):
    algo=choice(algorithm)
    if(algo=='ball_tree'):
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(metric))
    elif(algo=='kd_tree'):
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(sklearn.neighbors.KDTree.valid_metrics))
    else:
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(brute_metric))
    print("\n")
except:
  pass

the metrics used are n_neighbours=3, weights=uniform, algorithm=ball_tree, metrics=chebyshev
KNN with TFIDF accuracy = 57.59162303664922%
Cross Validation Accuracy: 0.56
[0.54509804 0.54901961 0.58661417]


the metrics used are n_neighbours=7, weights=uniform, algorithm=brute, metrics=cosine
KNN with TFIDF accuracy = 70.15706806282722%
Cross Validation Accuracy: 0.74
[0.70196078 0.73333333 0.79527559]


the metrics used are n_neighbours=3, weights=distance, algorithm=brute, metrics=hamming
KNN with TFIDF accuracy = 54.973821989528794%
Cross Validation Accuracy: 0.56
[0.52156863 0.62745098 0.53149606]


the metrics used are n_neighbours=11, weights=distance, algorithm=kd_tree, metrics=cityblock
KNN with TFIDF accuracy = 68.58638743455498%
Cross Validation Accuracy: 0.60
[0.62352941 0.58039216 0.5984252 ]


the metrics used are n_neighbours=7, weights=uniform, algorithm=ball_tree, metrics=braycurtis
KNN with TFIDF accuracy = 70.15706806282722%
Cross Validation Accuracy: 0.74
[0.7254902  

# Section 4: SPAM TEXT DATASET
Now let's use what we've learnt to classify texts as spam or not spam.

In [83]:
# Upload the spam text data CSV file that has been shared with you. You can also download the file from https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv


In [84]:
import pandas as pd
df = pd.read_csv('spam.csv', error_bad_lines=False)
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ã¼ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [85]:
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

In [86]:
df.head(5)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [87]:
len(df)

5572

In [88]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 2: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn(neigh=5,weigh='uniform',algo='auto',metri='euclidian'):
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

    knn = neighbors.KNeighborsClassifier(n_neighbors=neigh, weights=weigh, algorithm=algo, leaf_size=30, p=2, metric=metri, metric_params=None, n_jobs=1)
    print(f"the metrics used are n_neighbours={neigh}, weights={weigh}, algorithm={algo}, metrics={metri}")
    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn(neigh=5,weigh='uniform',algo='auto',metri='euclidian'):
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

    knn = neighbors.KNeighborsClassifier(n_neighbors=neigh, weights=weigh, algorithm=algo, leaf_size=30, p=2, metric=metri, metric_params=None, n_jobs=1)
    print(f"the metrics used are n_neighbours={neigh}, weights={weigh}, algorithm={algo}, metrics={metri}")
    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

In [91]:
try:
  for i in range(100):
    algo=choice(algorithm)
    if(algo=='ball_tree'):
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(metric))
    elif(algo=='kd_tree'):
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(sklearn.neighbors.KDTree.valid_metrics))
    else:
      predicted, y_test = bow_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(brute_metric))
except:
  pass

the metrics used are n_neighbours=11, weights=uniform, algorithm=ball_tree, metrics=rogerstanimoto
KNN with BOW accuracy = 89.05829596412556%
Cross Validation Accuracy: 0.88
[0.88290713 0.88223419 0.88215488]


the metrics used are n_neighbours=5, weights=distance, algorithm=ball_tree, metrics=russellrao
KNN with BOW accuracy = 95.24663677130046%
Cross Validation Accuracy: 0.95
[0.93808883 0.95356662 0.95353535]


the metrics used are n_neighbours=11, weights=distance, algorithm=ball_tree, metrics=jaccard
KNN with BOW accuracy = 98.56502242152466%
Cross Validation Accuracy: 0.98
[0.97375505 0.97913863 0.97239057]


the metrics used are n_neighbours=3, weights=distance, algorithm=kd_tree, metrics=infinity
KNN with BOW accuracy = 93.36322869955157%
Cross Validation Accuracy: 0.79
[0.53095559 0.91453567 0.91986532]


the metrics used are n_neighbours=3, weights=distance, algorithm=brute, metrics=rogerstanimoto
KNN with BOW accuracy = 94.7085201793722%
Cross Validation Accuracy: 0.94
[0.94

In [90]:
try:
  for i in range(100):
    algo=choice(algorithm)
    if(algo=='ball_tree'):
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(metric))
    elif(algo=='kd_tree'):
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(sklearn.neighbors.KDTree.valid_metrics))
    else:
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(brute_metric))
    print("\n")
except:
  pass

the metrics used are n_neighbours=5, weights=distance, algorithm=kd_tree, metrics=chebyshev
KNN with TFIDF accuracy = 91.74887892376682%
Cross Validation Accuracy: 0.82
[0.62920592 0.89771198 0.92053872]


the metrics used are n_neighbours=5, weights=distance, algorithm=kd_tree, metrics=manhattan
KNN with TFIDF accuracy = 93.90134529147983%
Cross Validation Accuracy: 0.93
[0.93001346 0.92261104 0.92592593]


the metrics used are n_neighbours=5, weights=distance, algorithm=brute, metrics=rogerstanimoto
KNN with TFIDF accuracy = 94.34977578475336%
Cross Validation Accuracy: 0.93
[0.93472409 0.92597577 0.93198653]


the metrics used are n_neighbours=11, weights=distance, algorithm=kd_tree, metrics=euclidean
KNN with TFIDF accuracy = 92.10762331838565%
Cross Validation Accuracy: 0.91
[0.91386272 0.9064603  0.91851852]


the metrics used are n_neighbours=7, weights=uniform, algorithm=brute, metrics=precomputed


In [ ]:
try:
  for i in range(50):
    algo=choice(algorithm)
    if(algo=='ball_tree'):
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(metric))
    elif(algo=='kd_tree'):
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(sklearn.neighbors.KDTree.valid_metrics))
    else:
      predicted, y_test = tfidf_knn(neigh=choice(neighbours),weigh=choice(weights),algo=algo,metri=choice(brute_metric))
    print("\n")
except:
  pass

the metrics used are n_neighbours=5, weights=uniform, algorithm=ball_tree, metrics=sokalsneath
KNN with TFIDF accuracy = 98.20627802690582%
Cross Validation Accuracy: 0.98
[0.97442799 0.97577389 0.97508418]


the metrics used are n_neighbours=11, weights=uniform, algorithm=brute, metrics=chebyshev
KNN with TFIDF accuracy = 91.21076233183857%
Cross Validation Accuracy: 0.72
[0.38627187 0.89165545 0.88215488]


the metrics used are n_neighbours=5, weights=uniform, algorithm=brute, metrics=sqeuclidean
KNN with TFIDF accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.90
[0.90040377 0.89771198 0.9010101 ]


the metrics used are n_neighbours=11, weights=distance, algorithm=brute, metrics=nan_euclidean
KNN with TFIDF accuracy = 92.10762331838565%
Cross Validation Accuracy: 0.91
[0.91386272 0.9064603  0.91851852]


the metrics used are n_neighbours=5, weights=uniform, algorithm=kd_tree, metrics=l1
KNN with TFIDF accuracy = 91.74887892376682%
Cross Validation Accuracy: 0.90
[0.89771198 

### Questions to Think About and Answer
1. Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
ans:becuase Tfidf accounts for the rarity of the words

2. Can you think of techniques that are better than both BoW and TF-IDF ?
ans: word2Vec, glove etc are better techniques than the above ones

3. Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.


ans:
Stemming:
Pros:
Stemming is a simpler process compared to lemmatization. It involves removing prefixes or suffixes to obtain the root form of a word.
Stemming is generally faster than lemmatization because it involves less complex linguistic analysis.

Reduced Dimensionality: Since stemming involves truncating words to their stems, it can reduce the dimensionality of the data, which might be beneficial in certain applications, such as text classification.

Cons:
Overstemming or Understemming: Stemming may lead to overstemming (reducing words to the same stem that are not actually related) or understemming (leaving words with different meanings in their original form).

Lack of Semantic Understanding: Stemming does not consider the context or meaning of words; it only focuses on the removal of affixes. This may lead to loss of semantic information.

Lemmatization:
Pros:
Linguistic Accuracy: Lemmatization provides a more linguistically accurate base form of a word by considering its meaning and context. It uses language dictionaries and morphological analysis.

Improved Information Retrieval: In tasks where understanding the context and meaning of words is crucial, such as information retrieval or question answering, lemmatization can be more effective.

Better for Downstream NLP Tasks: Lemmatization is often preferred for tasks like text summarization, sentiment analysis, and machine translation, where preserving the semantic meaning is essential.

Cons:
Computational Complexity: Lemmatization is more computationally intensive than stemming because it involves dictionary lookups and morphological analysis.

Dependency on Language Resources: Lemmatization relies on linguistic resources, such as dictionaries or language corpora, which may not be available for all languages or domains.

Potential for Ambiguity: In some cases, a word may have multiple lemmatization options, and choosing the correct one may depend on context.

### Useful Resources for further reading
1. Stemming and Lemmatization: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
2. TF-IDF and BoW : https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/
3. TF-IDF: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html


In [ ]:
#